<a href="https://colab.research.google.com/github/meghanabhange/DistrictAnalysisCovidIndia/blob/master/_notebooks/2020-04-01-DistrictAnalysisAssam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assam District Analysis
> plotting.

- toc: true 
- badges: true
- comments: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [0]:
#hide
# !pip install seirsplus
# !pip install geopandas
# !pip install geopatra
import json
import random
from pathlib import Path
import pandas as pd
import contextlib
import io
from seirsplus.models import *
import networkx
import sys
from ipywidgets import HBox, VBox, IntSlider, interactive, SelectionSlider, Label, Layout, HTML, Output, GridBox,FloatSlider,FloatLogSlider
from IPython.display import display
import folium
import json
import pandas as pd
import geopandas as gpd
import geopatra
import random
from IPython.utils import io
from branca.colormap import linear
from typing import Union, List

In [0]:
#hide
assam_travel_history = Path("AssamTravelHistory.xlsx").resolve()
xl = pd.ExcelFile(assam_travel_history)

In [0]:
#hide
def read_assam_excel_to_df(filename:str)->pd.DataFrame:
    xl = pd.ExcelFile(filename)
    df_list = []
    for sheet_name in xl.sheet_names:
        district_df = xl.parse(sheet_name)
        district_df["district"] = sheet_name
        district_df.drop(columns=["S.No."], inplace=True)
        df_list.append(district_df)
    return pd.concat(df_list, sort=False)

df = read_assam_excel_to_df(assam_travel_history)

In [0]:
#hide
df["DateOfArrival"] = pd.to_datetime(df["Date of arrival"])
df["DateOfReturn"] = pd.to_datetime(df["Date of Receipt"])
df.drop(columns=["Date of arrival", "Date of Receipt"], inplace=True)

In [0]:
#hide
df_copy = df.copy()
df_copy["Inflow"] = 1
assam_traveller_count_df = df_copy.groupby("district").agg({"Inflow": "sum"})
assam_traveller_count_df.reset_index(inplace=True)

In [0]:
#hide
def clean_district_names(dname: str):
    input_to_output_mapping = {
        "Cacher": "Cachar",
        "Kamrup_M": "Kamrup Metropolitan",
        "Kamrup_R": "Kamrup",
        "KarbiAnlong": "Karbi Anglong",
        "Baksha": "Baksa",
    }
    return input_to_output_mapping.get(dname, dname)

In [0]:
#hide
assam_traveller_count_df["district"] = assam_traveller_count_df.district.apply(clean_district_names)

In [0]:
#hide
assam_pop_web_raw = pd.read_html("https://www.census2011.co.in/census/state/districtlist/assam.html")
assam_pop_web_raw = assam_pop_web_raw[0][["District", "Population"]]
assam_pop_df = assam_pop_web_raw[~(assam_pop_web_raw["District"].apply(lambda x: len(x)) > 21)]
# assam_pop_df

In [0]:
#hide
assam_pop_df_rename = assam_pop_df.rename(columns = {'District':'district'}) 

In [0]:
#hide
assam_df = pd.merge(assam_pop_df_rename, assam_traveller_count_df, on="district", how="left")

# Running SIERS Plus for Assam
## Challenge: Assam has zero cases right now

In [0]:
#hide
@contextlib.contextmanager
def nostdout():
    save_stdout = sys.stdout
    sys.stdout = io.BytesIO()
    yield
    sys.stdout = save_stdout

In [0]:
#hide
def get_infections(
    initI: int = 100, initN: int = 10 ** 5, days_N: int = 21, beta: float = 2.4
) -> List[int]:
    model = SEIRSModel(
        beta=beta,
        sigma=1 / 5.2,
        gamma=1 / 12.39,
        initN=initN,
        initI=initI,
        beta_D=0.000,
        mu_D=0.02,
        #         sigma_D=1 / 4.0,
        #         gamma_D=1 / 9.0,
        theta_E=0.0002,
        theta_I=0.002,
        psi_E=0.2,
        psi_I=1.0,
    )
    with io.capture_output() as captured:
        model.run(T=days_N)
    S = model.numS  # time series of S counts
    E = model.numE  # time series of E counts
    I = model.numI  # time series of I counts
    D_E = model.numD_E  # time series of D_E counts
    D_I = model.numD_I  # time series of D_I counts
    R = model.numR  # time series of R counts
    F = model.numF  # time series of F counts
    t = model.tseries  # time values corresponding to the above time series
    return {"detected_exposed": D_E, "detected_infected": D_I, "model": model, "t": t}

In [0]:
#hide
def get_risk_estimates(
    assam_estimated_df: pd.DataFrame, beta
) -> Union[List, List]:
    days_N = 21
    atrisk_day14, atrisk_day21 = [], []
    for row in assam_estimated_df[["initI", "initN", "district"]].iterrows():
        initI, initN, district = row[1][0], int(row[1][1]), row[1][2]
        #     print(type(initI), type(initN))
        infection_results = get_infections(
            initI=initI, initN=initN, days_N=days_N, beta=beta
        )
        detected_infected = infection_results["detected_infected"]
        day14 = int(14 * len(detected_infected) / days_N)
        case_count_day14 = int(infection_results["detected_infected"][day14])
        case_count_day21 = int(infection_results["detected_infected"][-1])
        atrisk_day14.append(case_count_day14)
        atrisk_day21.append(case_count_day21)
    return atrisk_day14, atrisk_day21

In [0]:
#hide
def get_color_map(values):
    percent_travellers_infected = values['percent_travellers_infected'][0]
    unknown_to_known_travelers = values['unknown_to_known_travelers'][0]
    beta = values['beta'][0]
    
    assam_estimated_df = assam_df
    assam_estimated_df["initI"] = (
        percent_travellers_infected * unknown_to_known_travelers * assam_df["Inflow"]
    )
    assam_estimated_df["initI"] = assam_estimated_df["initI"].astype(int)
    assam_estimated_df["initN"] = assam_estimated_df["Population"].astype(int)
    atrisk_day14, atrisk_day21 = get_risk_estimates(assam_estimated_df, beta=beta)
    assam_estimated_df[f"day14"] = atrisk_day14
    assam_estimated_df[f"day21"] = atrisk_day21
    
    mins = assam_estimated_df['day21'].max()
    
    percent_travellers_infected = values['percent_travellers_infected'][1]
    unknown_to_known_travelers = values['unknown_to_known_travelers'][1]
    beta = values['beta'][1]

    assam_estimated_df = assam_df.copy()
    assam_estimated_df["initI"] = (
        percent_travellers_infected * unknown_to_known_travelers * assam_df["Inflow"]
    )
    assam_estimated_df["initI"] = assam_estimated_df["initI"].astype(int)
    assam_estimated_df["initN"] = assam_estimated_df["Population"].astype(int)
    atrisk_day14, atrisk_day21 = get_risk_estimates(assam_estimated_df, beta=beta)
    assam_estimated_df[f"day14"] = atrisk_day14
    assam_estimated_df[f"day21"] = atrisk_day21
    
    maxs = assam_estimated_df['day21'].min()

    return linear.OrRd_09.scale(mins, maxs)

In [0]:
#hide
def test(percent_travellers_infected, unknown_to_known_travelers, beta):
    
    display(
        Label(value='', layout = Layout(height="50"))
    )
        
    assam_estimated_df = assam_df.copy()
    assam_estimated_df["initI"] = (
        percent_travellers_infected * unknown_to_known_travelers * assam_df["Inflow"]
    )
    assam_estimated_df["initI"] = assam_estimated_df["initI"].astype(int)
    assam_estimated_df["initN"] = assam_estimated_df["Population"].astype(int)
    
    atrisk_day14, atrisk_day21 = get_risk_estimates(assam_estimated_df, beta=beta)
    assam_estimated_df[f"day14"] = atrisk_day14
    assam_estimated_df[f"day21"] = atrisk_day21
    
    geo_df = gpd.read_file("assam.json")
    df = gpd.GeoDataFrame(pd.merge(assam_estimated_df, geo_df, on="district"), geometry='geometry')
    
    map = df.folium.chloropeth(width=1200, height= 600,color_by="day21", color=color_map, style = {'color': 'black',
                    'weight': 1.5,
                    'dashArray': '10, 5',
                    'fillOpacity': 0.9,
                  }, zoom=7, tiles='cartodbdark_matter', tooltip=['district','day21'], location=(26.244156, 92.537842))
    
    present = pd.DataFrame(
        data = {
            'District':df['district'].tolist(),
            'AtRisk':df['day21'].tolist()
        }
    )
    
    widget1 = Output()
    widget2 = Output()
    
    with widget1:
        display(present)
    
    with widget2:
        display(map)
        
    hbox = GridBox([widget1, widget2], layout=Layout(grid_template_columns='300px 1000px'))
    
    display(hbox)

In [0]:
#hide
slider_layout = Layout(width='600px',height='20px')
style_dict = {'description_width': '200px'}

float_value = [10**i for i in range(-10,0)]
float_selections = [("%g"%i,i) for i in float_value]

values ={
        'percent_travellers_infected' : (1e-10,1e-1),
        'unknown_to_known_travelers' : (5,50),
        'beta' : (0.5, 5.0)
    }

color_map = get_color_map(values)

a = interactive(
    test,
    percent_travellers_infected = SelectionSlider(
        options=float_selections,
        value=1e-3,
        description="percent_travellers_infected: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    unknown_to_known_travelers = IntSlider(
        value=10, 
        min=5, 
        max=50,
        step=5,
        description="unknown_to_known_travelers: ",
        style = style_dict,
        layout = slider_layout
    ),
    
    beta = FloatSlider(
        value=2.4,
        min=0.5,
        max=5.0,
        step=0.01,
        description="beta: ",
        style = style_dict,
        layout = slider_layout
    )
)

In [17]:
display(a)

interactive(children=(SelectionSlider(description='percent_travellers_infected: ', index=7, layout=Layout(heig…